In [1]:
import time
import win32api,win32gui,win32con, win32ui
from ctypes import *
import cv2
import sys
import numpy as np
import os
from PIL import Image
import re
import pytesseract
import pyautogui
import cv2
import numpy as np

In [2]:
VK_CODE = {'backspace':0x08,
           'tab':0x09,
           'clear':0x0C,
           'enter':0x0D,
           'shift':0x10,
           'ctrl':0x11,
           'alt':0x12,
           'pause':0x13,
           'caps_lock':0x14,
           'esc':0x1B,
           'spacebar':0x20,
           ' ':0x20,
           'page_up':0x21,
           'page_down':0x22,
           'end':0x23,
           'home':0x24,
           'left_arrow':0x25,
           'up_arrow':0x26,
           'right_arrow':0x27,
           'down_arrow':0x28,
           'select':0x29,
           'print':0x2A,
           'execute':0x2B,
           'print_screen':0x2C,
           'ins':0x2D,
           'del':0x2E,
           'help':0x2F,
           '0':0x30,
           '1':0x31,
           '2':0x32,
           '3':0x33,
           '4':0x34,
           '5':0x35,
           '6':0x36,
           '7':0x37,
           '8':0x38,
           '9':0x39,
           'a':0x41,
           'b':0x42,
           'c':0x43,
           'd':0x44,
           'e':0x45,
           'f':0x46,
           'g':0x47,
           'h':0x48,
           'i':0x49,
           'j':0x4A,
           'k':0x4B,
           'l':0x4C,
           'm':0x4D,
           'n':0x4E,
           'o':0x4F,
           'p':0x50,
           'q':0x51,
           'r':0x52,
           's':0x53,
           't':0x54,
           'u':0x55,
           'v':0x56,
           'w':0x57,
           'x':0x58,
           'y':0x59,
           'z':0x5A,
           'numpad_0':0x60,
           'numpad_1':0x61,
           'numpad_2':0x62,
           'numpad_3':0x63,
           'numpad_4':0x64,
           'numpad_5':0x65,
           'numpad_6':0x66,
           'numpad_7':0x67,
           'numpad_8':0x68,
           'numpad_9':0x69,
           'multiply_key':0x6A,
           'add_key':0x6B,
           'separator_key':0x6C,
           'subtract_key':0x6D,
           'decimal_key':0x6E,
           'divide_key':0x6F,
           'F1':0x70,
           'F2':0x71,
           'F3':0x72,
           'F4':0x73,
           'F5':0x74,
           'F6':0x75,
           'F7':0x76,
           'F8':0x77,
           'F9':0x78,
           'F10':0x79,
           'F11':0x7A,
           'F12':0x7B,
           'F13':0x7C,
           'F14':0x7D,
           'F15':0x7E,
           'F16':0x7F,
           'F17':0x80,
           'F18':0x81,
           'F19':0x82,
           'F20':0x83,
           'F21':0x84,
           'F22':0x85,
           'F23':0x86,
           'F24':0x87,
           'num_lock':0x90,
           'scroll_lock':0x91,
           'left_shift':0xA0,
           'right_shift ':0xA1,
           'left_control':0xA2,
           'right_control':0xA3,
           'left_menu':0xA4,
           'right_menu':0xA5,
           'browser_back':0xA6,
           'browser_forward':0xA7,
           'browser_refresh':0xA8,
           'browser_stop':0xA9,
           'browser_search':0xAA,
           'browser_favorites':0xAB,
           'browser_start_and_home':0xAC,
           'volume_mute':0xAD,
           'volume_Down':0xAE,
           'volume_up':0xAF,
           'next_track':0xB0,
           'previous_track':0xB1,
           'stop_media':0xB2,
           'play/pause_media':0xB3,
           'start_mail':0xB4,
           'select_media':0xB5,
           'start_application_1':0xB6,
           'start_application_2':0xB7,
           'attn_key':0xF6,
           'crsel_key':0xF7,
           'exsel_key':0xF8,
           'play_key':0xFA,
           'zoom_key':0xFB,
           'clear_key':0xFE,
           '+':0xBB,
           ',':0xBC,
           '-':0xBD,
           '.':0xBE,
           '/':0xBF,
           '`':0xC0,
           ';':0xBA,
           '[':0xDB,
           '\\':0xDC,
           ']':0xDD,
           "'":0xDE,
'`':0xC0}

In [3]:
def clickLeftCur():
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN|win32con.MOUSEEVENTF_LEFTUP, 0, 0)
def doubleclickLeftCur():
    clickLeftCur()
    clickLeftCur()

def moveCurPos(x,y):
    windll.user32.SetCursorPos(x, y)

def getCurPos():
    return win32gui.GetCursorPos()

def window_capture(filename):
    hwnd = 0 # 視窗的編號，0號表示當前活躍視窗
    # 根據視窗控制代碼獲取視窗的裝置上下文DC（Divice Context）
    hwndDC = win32gui.GetWindowDC(hwnd)
    # 根據視窗的DC獲取mfcDC
    mfcDC = win32ui.CreateDCFromHandle(hwndDC)
    # mfcDC建立可相容的DC
    saveDC = mfcDC.CreateCompatibleDC()
    # 建立bigmap準備儲存圖片
    saveBitMap = win32ui.CreateBitmap()
    # 獲取監控器資訊
    MoniterDev = win32api.EnumDisplayMonitors(None, None)
    w = MoniterDev[0][2][2]
    h = MoniterDev[0][2][3]
    # print w,h　　　#圖片大小
    # 為bitmap開闢空間
    saveBitMap.CreateCompatibleBitmap(mfcDC, w, h)
    # 高度saveDC，將截圖儲存到saveBitmap中
    saveDC.SelectObject(saveBitMap)
    # 擷取從左上角（0，0）長寬為（w，h）的圖片
#     w = 1640
#     h = 289
#     saveDC.BitBlt((260, 572), (w, h), mfcDC, (260, 572), win32con.SRCCOPY)
    saveDC.BitBlt((0,0), (w, h), mfcDC, (0,0), win32con.SRCCOPY)
    saveBitMap.SaveBitmapFile(saveDC, filename)
    
def typeword(word):
    dict = {}
    #處理A~Z
    for w in range(ord('A'), ord('Z') + 1):
        dict[chr(w)] = w
        
    #處理0~9
    for w in range(ord('0'), ord('9') + 1):
        dict[chr(w)] = w
    
    #處理其他
    dict['/'] = 111
    dict['enter'] = 108
    dict[':'] = 0xBA
    for i in word:
        if i == ':':
            win32api.keybd_event(VK_CODE['left_shift'], 0,0,0)
            win32api.keybd_event(VK_CODE[';'], 0,0,0)
            time.sleep(.05)
            win32api.keybd_event(VK_CODE['left_shift'],0 ,win32con.KEYEVENTF_KEYUP ,0)
            win32api.keybd_event(VK_CODE[';'],0 ,win32con.KEYEVENTF_KEYUP ,0)
        else:
            win32api.keybd_event(VK_CODE[i],0,0,0)  
        time.sleep(0.2)
    

In [4]:
# #獲取滑鼠當前位置
xy=getCurPos()
print(xy)

(573, 570)


In [5]:
#virtual button
kd_number = '#2006'
app_icon = (173, 1058)
zoom_out = (158, 885)
chat_lt = (479, 180)
#chat_rb = (1145, 897)
chat_rb = (1141, 959)
# city_position1 = (755, 548)
city_position1 = (688,533)
city_position2 = (722,550)
crown = (903, 333) #(903, 341) to (957, 394)
#judge = (427, 586)
#duke = (695, 588)
duke = (426, 587) #fake
architect = (957, 582)
scientist = (1227, 588)
confirm = (817, 862)
chat_button = (261, 912)
chat_bar = (796, 933)
search_coord = (597, 148)
search_x = (835, 299)
search_y = (1019, 301)
search_button = (1128, 293)
close_chat = (1181, 542)
home = (171, 882)
chat_lt = (479, 180)
chat_rb = (1145, 897)
chat_window = (679, 926)
chat_random = (793, 158)
#other variable
tesseract_path = 'C:/Program Files/Tesseract-OCR/tesseract'
crown_img_path = 'crown.png'
lt_for_check = (223, 309)
rb_for_check = (1475, 752)

In [6]:
def find_subarray(a,b):
    for i in range(a.shape[0]-b.shape[0]+1):
        for j in range(a.shape[1]-b.shape[1]+1):
            if np.array_equal(a[i:i+b.shape[0],j:j+b.shape[1]], b):
                #print('Found at index of (', i, ',', j, ')')
                return (i,j)
    return crown

def trim(origin, target):
    if target in origin:
        return (trim(origin[origin.index(target)+1:],target))
    else:
        return (origin)
    
def  action(x,y,title):
    title_position = {'DUKE':duke, 'ARCH':architect, 'SCIE':scientist}


    moveCurPos(close_chat[0], close_chat[1])
    clickLeftCur()
    time.sleep(1)

    moveCurPos(search_coord[0], search_coord[1])
    clickLeftCur()
    time.sleep(1)

    moveCurPos(search_x[0], search_x[1])
    clickLeftCur()
    time.sleep(1)

    typeword(x)
    typeword(['enter'])

    moveCurPos(search_y[0], search_y[1])
    clickLeftCur()
    time.sleep(1)

    typeword(y)
    typeword(['enter'])

    moveCurPos(search_button[0], search_button[1])
    clickLeftCur()
    time.sleep(2)

    moveCurPos(city_position1[0], city_position1[1])
    clickLeftCur()
    time.sleep(1)

    
    #crown position depends
    img = pyautogui.screenshot(region=[lt_for_check[0],lt_for_check[1],rb_for_check[0]-lt_for_check[0],rb_for_check[1]-lt_for_check[1]]) # x,y,w,h
    img = cv2.cvtColor(np.asarray(img), cv2.COLOR_BGR2GRAY)
    img_crown = cv2.imread(crown_img_path)
    img_crown = cv2.cvtColor(img_crown, cv2.COLOR_BGR2GRAY)

    crown = find_subarray(img,img_crown)
   
        
    moveCurPos(crown[0], crown[1])
    clickLeftCur()
    time.sleep(1)

    moveCurPos(title_position[title][0], title_position[title][1])
    clickLeftCur()
    time.sleep(1)

    moveCurPos(confirm[0], confirm[1])
    clickLeftCur()
    time.sleep(1)

    moveCurPos(home[0], home[1])
    clickLeftCur()
    time.sleep(1)

    moveCurPos(home[0], home[1])
    clickLeftCur()
    time.sleep(2)

    moveCurPos(chat_button[0], chat_button[1])
    clickLeftCur()
    time.sleep(1)

In [7]:
#nain function
if __name__ == '__main__':
    ##### Action
    moveCurPos(app_icon[0], app_icon[1])
    clickLeftCur()
    time.sleep(1)

    current_using_title = {'DUKE':{'coord':[], 'time':0}, 'ARCH':{'coord':[], 'time':0}, 'SCIE':{'coord':[], 'time':0}}
    lineup = {'DUKE':[], 'ARCH':[], 'SCIE':[]} #
    lineup_old = {'DUKE':[], 'ARCH':[], 'SCIE':[]} #
    while True:

        ##CHAT ROOM SCAN##

        ##1. screenshot
        img = pyautogui.screenshot(region=[chat_lt[0],chat_lt[1],chat_rb[0]-chat_lt[0],chat_rb[1]-chat_lt[1]]) # x,y,w,h
        img = cv2.cvtColor(np.asarray(img),cv2.COLOR_RGB2BGR)

        ##2. screenshot to text
        pytesseract.pytesseract.tesseract_cmd = tesseract_path
        for i in range(img.shape[0]):
            if np.array_equal(img[0-i][575],[229,117,14]):
                crop_y = 0-i
    #             print(crop_y)
                break
        img = Image.fromarray(img[crop_y:,])
        text = pytesseract.image_to_string(img, lang='eng')
        #print(text)

        ##3. put text from screenshot to lineup
        #clear content over division line
        increase_num = 0
#         if 'LOAD' in text:
#             text = trim(text,'LOAD')
        print('1. Text after trim:\n',text)
        li = list(filter(None, re.split(' |\n|]|\)', text)))
        for i in range(len(li)):
            if kd_number in li[i]:
                if ('DUKE' in li[i-1]) or ('ARCH' in li[i-1]) or ('SCIE' in li[i-1]):
                    for j in [li[i+1], li[i+2], li[i+3], [i+4]]:
                        if 'X:' in j:
                            x_coord = j[2:]
                        if ('Y:' in j) or ('¥' in j):
                            y_coord = j[2:]
                    lineup[li[i-1]].append((x_coord, y_coord))
                    increase_num += 1
        print('2. lineup:',lineup)

        #Type division line##
        if increase_num > 0:
            moveCurPos(chat_window[0], chat_window[1])
            clickLeftCur()
            time.sleep(1)
    #         typeword('-- l o a d i n g --')

            #paste
            win32api.keybd_event(VK_CODE['left_control'], 0,0,0)
            win32api.keybd_event(VK_CODE['v'], 0,0,0)
            time.sleep(.05)
            win32api.keybd_event(VK_CODE['left_control'],0 ,win32con.KEYEVENTF_KEYUP ,0)
            win32api.keybd_event(VK_CODE['v'],0 ,win32con.KEYEVENTF_KEYUP ,0)

            typeword(['enter'])
            moveCurPos(chat_random[0], chat_random[1])
            clickLeftCur()
            time.sleep(1)

        ##UPDATE CURRENT USING TITLE LIST##
        #case 1: times up (duke: 3m, arch & sci: 5m)
        for index in current_using_title.keys():
            if current_using_title[index]['time'] == -1:
                pass
            else:
                if time.time() - current_using_title[index]['time'] >= 60:
                    current_using_title[index]['coord'] = []
                    current_using_title[index]['time'] = 0
                if len(current_using_title[index]['coord']) == 0:
                    print(index,'is available now !')
                else:
                    print('Left time of', index, ':', time.time()-current_using_title[index]['time'], '(s)')

        #case 2: done

        print('3. Current using:', current_using_title)



        ##EXECUTION GIVING TITLE## *GIVE ONE AT A TIME*
        for index in lineup.keys():
            if len(lineup[index]) != 0:
                if len(current_using_title[index]['coord']) == 0:
                    current_using_title[index]['coord'] = lineup[index].pop(0)
                    current_using_title[index]['time'] = time.time()
                    action(current_using_title[index]['coord'][0],current_using_title[index]['coord'][1],index)
                    print('4. Giving title:', index, current_using_title[index]['coord'])
    
    

1. Text after trim:
 
2. lineup: {'DUKE': [], 'ARCH': [], 'SCIE': []}
DUKE is available now !
ARCH is available now !
SCIE is available now !
3. Current using: {'DUKE': {'coord': [], 'time': 0}, 'ARCH': {'coord': [], 'time': 0}, 'SCIE': {'coord': [], 'time': 0}}
1. Text after trim:
 
2. lineup: {'DUKE': [], 'ARCH': [], 'SCIE': []}
DUKE is available now !
ARCH is available now !
SCIE is available now !
3. Current using: {'DUKE': {'coord': [], 'time': 0}, 'ARCH': {'coord': [], 'time': 0}, 'SCIE': {'coord': [], 'time': 0}}
1. Text after trim:
 
2. lineup: {'DUKE': [], 'ARCH': [], 'SCIE': []}
DUKE is available now !
ARCH is available now !
SCIE is available now !
3. Current using: {'DUKE': {'coord': [], 'time': 0}, 'ARCH': {'coord': [], 'time': 0}, 'SCIE': {'coord': [], 'time': 0}}
1. Text after trim:
 
2. lineup: {'DUKE': [], 'ARCH': [], 'SCIE': []}
DUKE is available now !
ARCH is available now !
SCIE is available now !
3. Current using: {'DUKE': {'coord': [], 'time': 0}, 'ARCH': {'co

1. Text after trim:
 
2. lineup: {'DUKE': [], 'ARCH': [], 'SCIE': []}
DUKE is available now !
ARCH is available now !
SCIE is available now !
3. Current using: {'DUKE': {'coord': [], 'time': 0}, 'ARCH': {'coord': [], 'time': 0}, 'SCIE': {'coord': [], 'time': 0}}
1. Text after trim:
 
2. lineup: {'DUKE': [], 'ARCH': [], 'SCIE': []}
DUKE is available now !
ARCH is available now !
SCIE is available now !
3. Current using: {'DUKE': {'coord': [], 'time': 0}, 'ARCH': {'coord': [], 'time': 0}, 'SCIE': {'coord': [], 'time': 0}}
1. Text after trim:
 
2. lineup: {'DUKE': [], 'ARCH': [], 'SCIE': []}
DUKE is available now !
ARCH is available now !
SCIE is available now !
3. Current using: {'DUKE': {'coord': [], 'time': 0}, 'ARCH': {'coord': [], 'time': 0}, 'SCIE': {'coord': [], 'time': 0}}
1. Text after trim:
 
2. lineup: {'DUKE': [], 'ARCH': [], 'SCIE': []}
DUKE is available now !
ARCH is available now !
SCIE is available now !
3. Current using: {'DUKE': {'coord': [], 'time': 0}, 'ARCH': {'co

1. Text after trim:
 
2. lineup: {'DUKE': [], 'ARCH': [], 'SCIE': []}
DUKE is available now !
ARCH is available now !
SCIE is available now !
3. Current using: {'DUKE': {'coord': [], 'time': 0}, 'ARCH': {'coord': [], 'time': 0}, 'SCIE': {'coord': [], 'time': 0}}
1. Text after trim:
 
2. lineup: {'DUKE': [], 'ARCH': [], 'SCIE': []}
DUKE is available now !
ARCH is available now !
SCIE is available now !
3. Current using: {'DUKE': {'coord': [], 'time': 0}, 'ARCH': {'coord': [], 'time': 0}, 'SCIE': {'coord': [], 'time': 0}}
1. Text after trim:
 
2. lineup: {'DUKE': [], 'ARCH': [], 'SCIE': []}
DUKE is available now !
ARCH is available now !
SCIE is available now !
3. Current using: {'DUKE': {'coord': [], 'time': 0}, 'ARCH': {'coord': [], 'time': 0}, 'SCIE': {'coord': [], 'time': 0}}
1. Text after trim:
 
2. lineup: {'DUKE': [], 'ARCH': [], 'SCIE': []}
DUKE is available now !
ARCH is available now !
SCIE is available now !
3. Current using: {'DUKE': {'coord': [], 'time': 0}, 'ARCH': {'co

1. Text after trim:
 
2. lineup: {'DUKE': [], 'ARCH': [], 'SCIE': []}
DUKE is available now !
ARCH is available now !
SCIE is available now !
3. Current using: {'DUKE': {'coord': [], 'time': 0}, 'ARCH': {'coord': [], 'time': 0}, 'SCIE': {'coord': [], 'time': 0}}
1. Text after trim:
 [zl06] HOSE Oat

: ae A (1S

DUKE (#2006 X:523
Y:616)

 

 

2. lineup: {'DUKE': [('523', '616')], 'ARCH': [], 'SCIE': []}
DUKE is available now !
ARCH is available now !
SCIE is available now !
3. Current using: {'DUKE': {'coord': [], 'time': 0}, 'ARCH': {'coord': [], 'time': 0}, 'SCIE': {'coord': [], 'time': 0}}
4. Giving title: DUKE ('523', '616')
1. Text after trim:
  

2. lineup: {'DUKE': [], 'ARCH': [], 'SCIE': []}
Left time of DUKE : 23.83553910255432 (s)
ARCH is available now !
SCIE is available now !
3. Current using: {'DUKE': {'coord': ('523', '616'), 'time': 1603697142.492961}, 'ARCH': {'coord': [], 'time': 0}, 'SCIE': {'coord': [], 'time': 0}}
1. Text after trim:
 
2. lineup: {'DUKE': [], 'A

KeyboardInterrupt: 